# Ensamblar y calcular ganancia

### Autor: Federico Picado
### Fecha de última modificación: 28/10/2024
### Descripción:

Podemos evaluar la ganancia de un modelo en particular, ordenado de distinta forma.
Tambien podemos realizar ensambles entre modelos y evaluar su ganancia. 

## Parámetros

## Input

In [1]:
# Ya en el input se ejecutan cosas
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [2]:
base_path = '/home/fedepicado/'
modelos_path = base_path + 'buckets/b1/exp/'
predicciones_path = base_path + 'buckets/b1/predicciones/'
db_path = base_path + 'buckets/b1/db/'
dataset_path = base_path + 'buckets/b1/datasets/'
dataset_file = 'competencia_02.parquet'
full_path = dataset_path + dataset_file

# nombre de la carpeta donde se guardaron los resultados para cada modelo segun la metrica
dataset_folder1="modelo_base"


In [3]:
data = pd.read_parquet(dataset_path + dataset_file)

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [4]:
# en este ultimo jobs se evalua la performance del modelo.

# MAIN

In [5]:
# Crear un diccionario para almacenar los DataFrames
dataframes = {}

# Función para cargar los CSV de una carpeta específica y asignarlos a variables
def cargar_csv_de_carpeta(dataset_folder):
    folder_path = os.path.join(predicciones_path, dataset_folder)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            # Nombre de la variable sin extensión
            variable_name = file_name.replace('.csv', '')
            
            # Cargar el DataFrame
            file_path = os.path.join(folder_path, file_name)
            dataframes[variable_name] = pd.read_csv(file_path)
            print(f"Archivo '{file_name}' cargado en la variable '{variable_name}'")

# Cargar los archivos de cada carpeta
cargar_csv_de_carpeta(dataset_folder1)


Archivo 'predicciones_promedio.csv' cargado en la variable 'predicciones_promedio'


In [6]:
dataframes

{'predicciones_promedio':         numero_de_cliente  Probabilidad
 0              1316192480  9.997836e-01
 1              1054008891  9.996265e-01
 2              1457625184  9.996052e-01
 3              1455026133  9.995824e-01
 4               560453687  9.995650e-01
 ...                   ...           ...
 164871          992551575  5.251881e-14
 164872         1390248594  5.102531e-14
 164873         1348371838  4.534367e-14
 164874          757562128  2.636833e-14
 164875         1308959428  5.363520e-15
 
 [164876 rows x 2 columns]}

In [7]:
bajas = data.loc[data["foto_mes"] == 202106, ["numero_de_cliente", "clase_ternaria"]]

In [8]:
bajas["clase_ternaria"].value_counts()

clase_ternaria
CONTINUA    162894
BAJA+2        1074
BAJA+1         908
Name: count, dtype: int64

In [9]:
def calculoGanancia(bajas,prediccion,corte):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia para el público.
    ganancia_privado: Ganancia para el privado.
    '''
    # Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=123
    )

    # Clientes que decido estimular
    estimulos = prediccion.iloc[:corte] 

    # Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # 5. Calcular la ganancia para cada conjunto con normalización
    # Primero, calculamos la ganancia en cada conjunto
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Luego, normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado

In [10]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)

# Iteramos sobre cada modelo en el diccionario de predicciones
for model_name, pred_model in dataframes.items():
    # Aseguramos que las predicciones estén ordenadas por probabilidad descendente
    pred_model_sorted = pred_model.sort_values('Probabilidad', ascending=False)
    
    # Iteramos sobre cada corte
    for corte in cortes:
        ganancia_publico, ganancia_privado = calculoGanancia(bajas, pred_model_sorted, corte)
        
        # Almacenamos los resultados
        resultados.append({
            'Modelo': model_name,
            'Corte': corte,
            'Ganancia Público': ganancia_publico,
            'Ganancia Privado': ganancia_privado
        })

# Convertimos los resultados en un DataFrame
resultados = pd.DataFrame(resultados)

fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")


Tiempo de ejecución: 2.35 segundos


In [11]:
resultados

,Modelo,Corte,Ganancia Público,Ganancia Privado
0,predicciones_promedio,5000,6.304667e+07,74180000.0
1,predicciones_promedio,6000,6.785333e+07,76920000.0
2,predicciones_promedio,7000,6.925333e+07,83120000.0
3,predicciones_promedio,8000,7.380333e+07,88770000.0
4,predicciones_promedio,9000,7.585667e+07,90690000.0
5,predicciones_promedio,10000,7.905333e+07,92520000.0
6,predicciones_promedio,11000,7.875000e+07,91450000.0
7,predicciones_promedio,12000,8.225000e+07,88750000.0
8,predicciones_promedio,13000,8.026667e+07,87200000.0
9,predicciones_promedio,14000,7.959000e+07,83490000.0


In [12]:
# Pivotamos el DataFrame 'resultados' para reorganizar las ganancias
resultados_pivot = resultados.pivot_table(
    index='Corte',
    columns='Modelo',
    values=['Ganancia Público', 'Ganancia Privado']
)

# Aplanamos las columnas para facilitar el acceso
resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

# Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
# Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
columnas_ordenadas = []
for modelo in resultados['Modelo'].unique():
    columnas_ordenadas.append(f'Ganancia Público_{modelo}')
    columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

# Reorganizamos el DataFrame usando el nuevo orden de columnas
resultados_pivot = resultados_pivot[columnas_ordenadas]

# Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
resultados_pivot = resultados_pivot.reset_index()
resultados_pivot

,Corte,Ganancia Público_predicciones_promedio,Ganancia Privado_predicciones_promedio
0,5000,6.304667e+07,74180000.0
1,6000,6.785333e+07,76920000.0
2,7000,6.925333e+07,83120000.0
3,8000,7.380333e+07,88770000.0
4,9000,7.585667e+07,90690000.0
5,10000,7.905333e+07,92520000.0
6,11000,7.875000e+07,91450000.0
7,12000,8.225000e+07,88750000.0
8,13000,8.026667e+07,87200000.0
9,14000,7.959000e+07,83490000.0


## Tengo el problema que segun la semilla con la que dividi el publico privado me puede dar distinto.. hago semillero

In [13]:
def calculoGanancia(bajas, prediccion, corte, random_state):
    ''' 
    Calcula la ganancia para una semilla específica.
    
    Parámetros:
    bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    random_state: int, semilla para train_test_split.
    
    Retorna:
    ganancia_publico: Ganancia normalizada para el conjunto público.
    ganancia_privado: Ganancia normalizada para el conjunto privado.
    '''
    # Realizar el split en público y privado
    Publico, Privado = train_test_split(
        bajas,
        test_size=0.7,
        stratify=bajas['clase_ternaria'],
        random_state=random_state
    )

    # Obtener los estímulos según el modelo
    estimulos = prediccion.iloc[:corte] 

    # Obtener los estímulos en el conjunto público y privado
    estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
    estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

    # Calcular los verdaderos positivos en cada conjunto
    TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
    TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

    # Calcular la ganancia para cada conjunto con normalización
    ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
    ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

    # Normalizamos dividiendo por el porcentaje correspondiente
    ganancia_publico = ganancia_publico_sin_norm / 0.3
    ganancia_privado = ganancia_privado_sin_norm / 0.7

    return ganancia_publico, ganancia_privado

def calculoGananciaPromedio(bajas, prediccion, corte, n_seeds=100):
    '''
    Calcula la ganancia promedio para múltiples semillas.
    
    Parámetros:
    bajas: Df con columnas "numero_de_cliente" y "clase_ternaria".
    prediccion: Df con columnas "numero_de_cliente" y "Probabilidad".
    corte: int, cantidad de estímulos.
    n_seeds: int, número de semillas aleatorias (por defecto 200).
    
    Retorna:
    promedio_ganancia_publico: Ganancia promedio normalizada para el conjunto público.
    promedio_ganancia_privado: Ganancia promedio normalizada para el conjunto privado.
    '''
    
    seeds = np.random.randint(1000, 100001, size=n_seeds)
    ganancias_publico = []
    ganancias_privado = []

    for seed in seeds:
        ganancia_publico, ganancia_privado = calculoGanancia(bajas, prediccion, corte, random_state=seed)
        ganancias_publico.append(ganancia_publico)
        ganancias_privado.append(ganancia_privado)

    promedio_ganancia_publico = np.mean(ganancias_publico)
    promedio_ganancia_privado = np.mean(ganancias_privado)

    return promedio_ganancia_publico,promedio_ganancia_privado


In [14]:
inicio = time.time()

resultados = []
cortes = range(5000, 20000, 1000)

# Iteramos sobre cada modelo en el diccionario de predicciones
for model_name, pred_model in dataframes.items():
    # Aseguramos que las predicciones estén ordenadas por probabilidad descendente
    pred_model_sorted = pred_model.sort_values('Probabilidad', ascending=False)
    
    # Iteramos sobre cada corte
    for corte in cortes:
        ganancia_publico, ganancia_privado = calculoGananciaPromedio(bajas, pred_model_sorted, corte)
        
        # Almacenamos los resultados
        resultados.append({
            'Modelo': model_name,
            'Corte': corte,
            'Ganancia Público': ganancia_publico,
            'Ganancia Privado': ganancia_privado
        })

# Convertimos los resultados en un DataFrame
resultados = pd.DataFrame(resultados)

fin = time.time()
tiempo_ejecucion = fin - inicio

print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

Tiempo de ejecución: 228.50 segundos


In [15]:
# Pivotamos el DataFrame 'resultados' para reorganizar las ganancias
resultados_pivot = resultados.pivot_table(
    index='Corte',
    columns='Modelo',
    values=['Ganancia Público', 'Ganancia Privado']
)

# Aplanamos las columnas para facilitar el acceso
resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

# Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
# Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
columnas_ordenadas = []
for modelo in resultados['Modelo'].unique():
    columnas_ordenadas.append(f'Ganancia Público_{modelo}')
    columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

# Reorganizamos el DataFrame usando el nuevo orden de columnas
resultados_pivot = resultados_pivot[columnas_ordenadas]

# Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
resultados_pivot = resultados_pivot.reset_index()
resultados_pivot

,Corte,Ganancia Público_predicciones_promedio,Ganancia Privado_predicciones_promedio
0,5000,6.948783e+07,71419500.0
1,6000,7.281797e+07,74792300.0
2,7000,7.850850e+07,79153500.0
3,8000,8.439130e+07,84232300.0
4,9000,8.630627e+07,86211600.0
5,10000,8.932490e+07,88117900.0
6,11000,8.760733e+07,87654000.0
7,12000,8.622507e+07,87046400.0
8,13000,8.545390e+07,84976900.0
9,14000,8.237087e+07,82298200.0


In [16]:
bajas

,numero_de_cliente,clase_ternaria
29,896098966,CONTINUA
61,901145889,CONTINUA
93,902802426,CONTINUA
117,903624965,CONTINUA
142,904421628,BAJA+1
...,...,...
4735545,1593392552,CONTINUA
4735549,1593812536,CONTINUA
4735578,1549407028,CONTINUA
4735586,1557363225,CONTINUA


In [17]:
dataframes

{'predicciones_promedio':         numero_de_cliente  Probabilidad
 0              1316192480  9.997836e-01
 1              1054008891  9.996265e-01
 2              1457625184  9.996052e-01
 3              1455026133  9.995824e-01
 4               560453687  9.995650e-01
 ...                   ...           ...
 164871          992551575  5.251881e-14
 164872         1390248594  5.102531e-14
 164873         1348371838  4.534367e-14
 164874          757562128  2.636833e-14
 164875         1308959428  5.363520e-15
 
 [164876 rows x 2 columns]}

In [32]:
def generar_curvas_roc_multiples(dataframes, bajas, punto_corte):
    """
    Generates and plots ROC curves for multiple models using the entire dataset,
    and computes true positives at a specific cutoff.

    Args:
        dataframes (dict): Dictionary where keys are model names and values are DataFrames.
                           Each DataFrame must have columns ['numero_de_cliente', 'Probabilidad'].
        bajas (DataFrame): DataFrame with columns ['numero_de_cliente', 'clase_ternaria'].
                           'clase_ternaria' can be 'CONTINUA', 'BAJA+1', or 'BAJA+2'.
        punto_corte (int): Number of clients to select (e.g., 13000).

    Returns:
        dict: Dictionary with FPR, TPR, AUC, and true positives at cutoff for each model.
    """
    resultados = {}

    for model_name, pred_model in dataframes.items():
        # Merge predictions with actual classes
        merged_data = pred_model.merge(bajas, on='numero_de_cliente', how='inner')

        # Convert 'clase_ternaria' to binary variable
        merged_data['bajas_reales'] = (merged_data['clase_ternaria'] == 'BAJA+2').astype(int)

        # Compute ROC curve and AUC using all data
        fpr, tpr, thresholds = roc_curve(merged_data['bajas_reales'], merged_data['Probabilidad'])
        roc_auc = auc(fpr, tpr)

        # Sort by predicted probabilities
        merged_data_sorted = merged_data.sort_values('Probabilidad', ascending=False)

        # Select top 'punto_corte' clients
        top_clients = merged_data_sorted.head(punto_corte)

        # Compute true positives at the cutoff
        true_positives = top_clients['bajas_reales'].sum()
        total_positives = merged_data['bajas_reales'].sum()

        # Store results
        resultados[model_name] = {
            "FPR": fpr,
            "TPR": tpr,
            "AUC": roc_auc,
            "True Positives at Cutoff": true_positives,
            "Total Positives": total_positives
        }

        # Plot ROC curve
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')

    # Configure the general ROC plot
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random')
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title('ROC Curves by Model')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show()

    return resultados


{'predicciones_promedio': {'FPR': array([0.00000000e+00, 6.10493156e-06, 4.88394525e-05, ...,
         9.77546062e-01, 9.77546062e-01, 1.00000000e+00]),
  'TPR': array([0.       , 0.       , 0.       , ..., 0.9990689, 1.       ,
         1.       ]),
  'AUC': np.float64(0.881380560128949),
  'True Positives at Cutoff': np.int64(566),
  'Total Positives': np.int64(1074)}}

In [33]:
generar_curvas_roc_multiples(dataframes,bajas,10000)

In [26]:
def evaluar_modelos(dataframes, bajas, num_clientes):
    """
    Evalúa y grafica métricas para varios modelos, seleccionando los primeros N clientes.

    Args:
        dataframes (dict): Diccionario donde las claves son los nombres de los modelos y los valores son DataFrames.
                           Cada DataFrame debe tener las columnas ['numero_de_cliente', 'Probabilidad'].
        bajas (DataFrame): DataFrame con las columnas ['numero_de_cliente', 'clase_ternaria'].
                           `clase_ternaria` puede tener los valores 'CONTINUA', 'BAJA+1' y 'BAJA+2'.
        num_clientes (int): Número de clientes a seleccionar (por ejemplo, 13000).

    Returns:
        dict: Diccionario con las métricas calculadas para cada modelo.
    """
    resultados = {}

    for model_name, pred_model in dataframes.items():
        # Ordenar las predicciones por probabilidad descendente
        pred_model_sorted = pred_model.sort_values('Probabilidad', ascending=False)

        # Seleccionar los primeros 'num_clientes' clientes
        pred_model_filtrado = pred_model_sorted.head(num_clientes)

        # Unir con el DataFrame 'bajas' para obtener la clase real
        merged_data = pred_model_filtrado.merge(bajas, on='numero_de_cliente', how='inner')

        # Convertir 'clase_ternaria' en variable binaria: 1 si es 'BAJA+2', 0 en otro caso
        merged_data['bajas_reales'] = (merged_data['clase_ternaria'] == 'BAJA+2').astype(int)

        # Crear la columna de predicciones: Asumimos que todos los seleccionados son predichos como 'BAJA+2'
        merged_data['prediccion'] = 1  # Porque hemos seleccionado estos clientes para enviar el estímulo

        # Obtener las bajas reales y las predicciones
        y_true = merged_data['bajas_reales']
        y_pred = merged_data['prediccion']

        # Calcular métricas
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        f1 = f1_score(y_true, y_pred)

        # Almacenar resultados
        resultados[model_name] = {
            "Verdaderos Positivos (TP)": tp,
            "Falsos Positivos (FP)": fp,
            "Verdaderos Negativos (TN)": tn,
            "Falsos Negativos (FN)": fn,
            "Precisión": precision,
            "Sensibilidad (Recall)": recall,
            "Exactitud (Accuracy)": accuracy,
            "F1-Score": f1,
            "Total Clientes Seleccionados": num_clientes,
            "Total BAJA+2 Reales en Seleccionados": y_true.sum()
        }

    # Mostrar resultados
    for model_name, metrics in resultados.items():
        print(f"\nMétricas para el modelo: {model_name}")
        for metric_name, value in metrics.items():
            print(f"{metric_name}: {value}")

    return resultados

In [27]:
evaluar_modelos(dataframes, bajas, 10000)


Métricas para el modelo: predicciones_promedio
Verdaderos Positivos (TP): 566
Falsos Positivos (FP): 9434
Verdaderos Negativos (TN): 0
Falsos Negativos (FN): 0
Precisión: 0.0566
Sensibilidad (Recall): 1.0
Exactitud (Accuracy): 0.0566
F1-Score: 0.10713609691463184
Total Clientes Seleccionados: 10000
Total BAJA+2 Reales en Seleccionados: 566


{'predicciones_promedio': {'Verdaderos Positivos (TP)': np.int64(566),
  'Falsos Positivos (FP)': np.int64(9434),
  'Verdaderos Negativos (TN)': np.int64(0),
  'Falsos Negativos (FN)': np.int64(0),
  'Precisión': np.float64(0.0566),
  'Sensibilidad (Recall)': np.float64(1.0),
  'Exactitud (Accuracy)': np.float64(0.0566),
  'F1-Score': np.float64(0.10713609691463184),
  'Total Clientes Seleccionados': 10000,
  'Total BAJA+2 Reales en Seleccionados': np.int64(566)}}